<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 10:40:52
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: -1.42 L (-1.01%)
Current PnL: -22.65 L (-15.09%)
CY Booked + Current PnL: -10.75 L (-7.16%)
-------------------
Total profit:  1.30 L
Total loss:  -23.95 L
-------------------
Total Booked + Current PnL: 16.23 L (13.18%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.57%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.86 L (64.72%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.08,53.0,M-SC,1.26,85386.0,-15391.0,15480.0,2.26,-15.27,18.13,0.09,245.0,-0.99,0.62,12.18,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.47,53.0,X-LC,3.73,211816.0,7316.0,15971.0,-1.12,3.58,7.54,11.39,10.0,0.46,1.54,12.85,X40N,ATH,PHARMA
83,VOLTAS,1530.00,-1.08,51.0,X-MC,1.00,207420.0,15678.0,22090.0,0.31,8.18,10.65,19.69,99.0,0.71,1.51,15.53,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,50.0,X-LC,2.86,215436.0,5070.0,23720.0,0.44,2.41,11.01,13.68,37.0,0.21,1.56,19.33,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194622.0,-5516.0,26410.0,-0.31,-2.76,13.57,10.44,4.0,-0.21,1.41,2.65,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.00,11.22,64.0,X-SC,10.29,196866.0,5860.0,51835.0,2.57,3.07,26.33,30.21,157.0,0.11,1.43,24.59,X40N,NTT,FINANCE
66,SONACOMS,1006.00,-37.35,61.0,M-SC,12.93,78050.0,-23109.0,98000.0,2.35,-22.84,125.56,74.03,202.0,-0.24,0.57,10.06,AR,BTT,AUTO
31,HINDZINC,730.22,38.11,70.0,M-LC,8.69,225025.0,19949.0,92620.0,2.32,9.73,41.16,54.89,52.0,0.22,1.63,36.17,X5K,ATH,METALS
76,TTKPRESTIG,770.00,98.08,53.0,M-SC,1.26,85386.0,-15391.0,15480.0,2.26,-15.27,18.13,0.09,245.0,-0.99,0.62,12.18,OX40N,NTT,DURABLES
34,ICICIPRULI,790.00,-20.88,52.0,X-MC,1.93,136199.0,411.0,42344.0,1.52,0.30,31.09,31.48,107.0,0.01,0.99,12.46,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-52.51,12.0,X-LC,0.98,149320.0,-127203.0,237269.0,-37.75,-46.00,158.90,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
53,RAJOOENG,143.1,-42.97,40.0,H-SC,17.39,86330.0,-16170.0,56771.0,-3.51,-15.78,65.76,39.61,114.0,-0.28,0.63,5.43,AR,ATH,MISC
81,VALIANTORG,838.0,-280.91,24.0,H-SC,3.06,100504.0,-63101.0,184415.0,-3.24,-38.57,183.49,74.15,139.0,-0.34,0.73,28.83,XR,NTT,CHEMICALS
10,BANDHANBNK,400.0,-11.95,50.0,H-SC,2.98,220202.0,-58357.0,315792.0,-2.46,-20.95,143.41,92.42,151.0,-0.18,1.60,27.26,XY24,NTT,BANKS
23,DEN,75.0,91.71,40.0,M-SC,3.79,53856.0,-30608.0,66146.0,-2.21,-36.24,122.82,42.07,237.0,-0.46,0.39,14.37,AR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.0,43.54,49.0,M-MC,12.59,226495.0,1533.0,166111.0,-0.74,0.68,73.34,74.53,192.0,0.01,1.64,33.15,XY24,BTT,STEEL
54,RECLTD,446.0,45.36,43.0,M-LC,6.20,203802.0,632.0,41494.0,-0.15,0.31,20.36,20.74,55.0,0.02,1.48,5.87,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,106.53,60.0,H-SC,11.60,130302.0,-12177.0,31702.0,-0.98,-8.55,24.33,13.70,163.0,-0.38,0.95,53.79,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.08,53.0,M-SC,1.26,85386.0,-15391.0,15480.0,2.26,-15.27,18.13,0.09,245.0,-0.99,0.62,12.18,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,47.0,H-SC,2.05,220500.0,-49167.0,85510.0,-0.28,-18.23,38.78,13.47,138.0,-0.57,1.60,12.12,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,1.97,139966.0,-35937.0,77961.0,1.22,-20.43,55.70,23.89,149.0,-0.46,1.02,20.88,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.05,42.0,H-MC,7.02,101403.0,-30432.0,72655.0,-0.28,-23.08,71.65,32.03,98.0,-0.42,0.74,13.58,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203802.0,632.0,41494.0,-0.15,0.31,20.36,20.74,55.0,0.02,1.48,5.87,XY25,NTT,FINANCE
59,SAIL,228.00,43.54,49.0,M-MC,12.59,226495.0,1533.0,166111.0,-0.74,0.68,73.34,74.53,192.0,0.01,1.64,33.15,XY24,BTT,STEEL
31,HINDZINC,730.22,38.11,70.0,M-LC,8.69,225025.0,19949.0,92620.0,2.32,9.73,41.16,54.89,52.0,0.22,1.63,36.17,X5K,ATH,METALS
85,WIPRO,420.00,-12.89,52.0,M-LC,5.96,153264.0,2319.0,107132.0,0.84,1.54,69.90,72.51,53.0,0.02,1.11,7.65,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,55.0,M-LC,9.56,157509.0,16669.0,27816.0,-0.98,11.84,17.66,31.59,66.0,0.60,1.14,41.46,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.11,70.0,M-LC,8.69,225025.0,19949.0,92620.0,2.32,9.73,41.16,54.89,52.0,0.22,1.63,36.17,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,62.0,H-MC,12.71,192788.0,12980.0,99440.0,-1.19,7.22,51.58,62.52,88.0,0.13,1.40,43.77,XR,NTT,BANKS
85,WIPRO,420.00,-12.89,52.0,M-LC,5.96,153264.0,2319.0,107132.0,0.84,1.54,69.90,72.51,53.0,0.02,1.11,7.65,XR,NTT,IT
36,INDIAMART,4810.62,-55.39,38.0,H-SC,8.32,124778.0,1442.0,130181.0,-1.84,1.17,104.33,106.72,119.0,0.01,0.91,24.82,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.37,31.0,M-SC,3.39,87225.0,-4274.0,83021.0,-0.10,-4.67,95.18,86.07,223.0,-0.05,0.63,26.67,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-52.51,12.0,X-LC,0.98,149320.0,-127203.0,237269.0,-37.75,-46.00,158.90,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.45,30.0,X-MC,1.35,227042.0,-20689.0,95744.0,-1.20,-8.35,42.17,30.30,92.0,-0.22,1.65,3.83,X40,ATH,ELECTRICAL
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51509.0,-13497.0,164427.0,-1.38,-20.76,319.22,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,-15.75,32.0,X-MC,5.48,177448.0,-23058.0,120203.0,-0.48,-11.50,67.74,48.45,90.0,-0.19,1.29,2.33,X40N,ATH,FINANCE
5,ASIANPAINT,3465.66,-18.55,33.0,X-LC,5.33,205586.0,-46182.0,99401.0,-0.27,-18.34,48.35,21.13,27.0,-0.46,1.49,10.58,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.90,41.0,H-LC,0.92,154580.0,-31515.0,78990.0,0.46,-16.93,51.10,25.51,15.0,-0.40,1.12,13.91,AR,ATH,ELECTRICAL
72,TATAMOTORS,1065.00,-52.51,12.0,X-LC,0.98,149320.0,-127203.0,237269.0,-37.75,-46.00,158.90,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
83,VOLTAS,1530.00,-1.08,51.0,X-MC,1.00,207420.0,15678.0,22090.0,0.31,8.18,10.65,19.69,99.0,0.71,1.51,15.53,XY25,NTT,AC
28,HAVELLS,2069.16,-8.45,30.0,X-MC,1.35,227042.0,-20689.0,95744.0,-1.20,-8.35,42.17,30.30,92.0,-0.22,1.65,3.83,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.58,41.0,X-MC,1.46,136626.0,493.0,30632.0,0.70,0.36,22.42,22.86,91.0,0.02,0.99,13.02,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51509.0,-13497.0,164427.0,-1.38,-20.76,319.22,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-52.51,12.0,X-LC,0.98,149320.0,-127203.0,237269.0,-37.75,-46.00,158.90,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
82,VBL,671.64,-22.00,36.0,X-LC,10.53,280656.0,-35186.0,147850.0,-0.58,-11.14,52.68,35.67,5.0,-0.24,2.04,1.17,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,32.0,X-MC,5.48,177448.0,-23058.0,120203.0,-0.48,-11.50,67.74,48.45,90.0,-0.19,1.29,2.33,X40N,ATH,FINANCE
21,COLPAL,3767.14,-5.20,37.0,X-MC,7.66,221060.0,-42305.0,155648.0,-0.48,-16.06,70.41,43.04,84.0,-0.27,1.60,2.63,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51509.0,-13497.0,164427.0,-1.38,-20.76,319.22,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-40.75,36.0,X-SC,3.30,80812.0,-64348.0,140273.0,-0.49,-44.33,173.58,52.31,136.0,-0.46,0.59,8.17,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.58,37.0,X-SC,4.32,94919.0,-33751.0,79049.0,-0.47,-26.23,83.28,35.20,219.0,-0.43,0.69,8.95,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.05,54.0,X-SC,1.02,108959.0,6955.0,36970.0,0.60,6.82,33.93,43.06,122.0,0.19,0.79,25.10,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.23,108180.0,-19758.0,66888.0,-0.26,-15.44,61.83,36.84,143.0,-0.30,0.79,10.75,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.33,47.0,X-LC,12.25,281643.0,-64313.0,132541.0,-0.37,-18.59,47.06,19.73,1.0,-0.49,2.04,3.73,X40,ATH,IT
39,INFY,2275.00,-18.16,52.0,X-LC,7.50,318414.0,5526.0,166148.0,0.11,1.77,52.18,54.87,3.0,0.03,2.31,8.53,X40,BTT,IT
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194622.0,-5516.0,26410.0,-0.31,-2.76,13.57,10.44,4.0,-0.21,1.41,2.65,X40,NTT,FMCG
82,VBL,671.64,-22.00,36.0,X-LC,10.53,280656.0,-35186.0,147850.0,-0.58,-11.14,52.68,35.67,5.0,-0.24,2.04,1.17,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,40.0,H-LC,7.29,245280.0,-16339.0,135542.0,-0.73,-6.25,55.26,45.57,7.0,-0.12,1.78,4.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.71,80736.0,-32813.0,72864.0,-1.27,-28.90,90.25,35.27,268.0,-0.45,0.59,97.60,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,39.0,H-SC,9.91,89460.0,-8520.0,30121.0,-1.21,-8.70,33.67,22.05,152.0,-0.28,0.65,30.04,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,106.53,60.0,H-SC,11.60,130302.0,-12177.0,31702.0,-0.98,-8.55,24.33,13.70,163.0,-0.38,0.95,53.79,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.47,147600.0,-1620.0,145799.0,0.98,-1.09,98.78,96.62,208.0,-0.01,1.07,59.74,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.54,147431.0,-31640.0,97363.0,-1.93,-17.67,66.04,36.71,135.0,-0.32,1.07,46.28,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.47,147600.0,-1620.0,145799.0,0.98,-1.09,98.78,96.62,208.0,-0.01,1.07,59.74,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.54,147431.0,-31640.0,97363.0,-1.93,-17.67,66.04,36.71,135.0,-0.32,1.07,46.28,XR,NTT,BANKS
31,HINDZINC,730.22,38.11,70.0,M-LC,8.69,225025.0,19949.0,92620.0,2.32,9.73,41.16,54.89,52.0,0.22,1.63,36.17,X5K,ATH,METALS
63,SHALBY,327.00,1174.32,54.0,M-SC,22.95,173285.0,-7858.0,53320.0,-1.88,-4.34,30.77,25.10,235.0,-0.15,1.26,38.87,XY24,NTT,HEALTHCARE
67,STARHEALTH,761.00,15.63,62.0,H-SC,8.12,257747.0,-23376.0,156246.0,-1.23,-8.32,60.62,47.26,144.0,-0.15,1.87,38.09,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.44
1,25,43.98
2,50,75.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.47
LC    33.68
MC    21.81
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.70
X40      14.83
XY25     12.07
XR       10.98
X40N     10.11
AR        8.19
OX40N     7.69
X200      1.82
X5K       1.63
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.65
X-LC    22.57
X-MC    16.05
M-SC    12.71
M-LC     5.36
H-LC     4.72
X-SC     4.66
H-MC     3.81
M-MC     1.64
L-SC     1.45
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.96,-6.65,41.80
IT,13.06,-16.82,80.09
FINANCE,12.18,-12.69,63.77
MISC,6.95,-20.09,86.64
BANKS,6.47,-12.00,70.92
PAINTS,5.59,-19.88,37.75
ELECTRICAL,5.34,-12.01,51.31
INSURANCE,3.85,-4.24,43.20
AUTO,2.85,-46.47,116.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3343050.0,22
XR,1397142.0,14
AR,1201888.0,9
X40,795523.0,10
X40N,636635.0,8
OX40N,586556.0,10
XY25,519200.0,8
SR,267036.0,2
X5K,92620.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3467323.0,25
M-SC,1289664.0,15
X-LC,1128321.0,13
X-MC,1116283.0,11
X-SC,539442.0,6
H-MC,355295.0,3
H-LC,289441.0,3
M-LC,269062.0,4
L-SC,262209.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206312.0      6
           XR         826011.0      7
           AR         823210.0      5
M-SC       XY24       696958.0      6
X-MC       XY24       593183.0      4
X-LC       X40        491315.0      5
           XY24       305043.0      2
X-SC       X40N       295966.0      4
H-SC       OX40N      273674.0      4
           SR         267036.0      2
X-MC       X40        225159.0      4
X-LC       X40N       220466.0      3
H-LC       AR         214532.0      2
H-MC       XY24       183200.0      1
X-MC       XY25       177738.0      2
L-SC       XR         172321.0      2
M-MC       XY24       166111.0      1
X-SC       XY24       164427.0      1
M-SC       AR         164146.0      2
           OX40N      150339.0      4
           XY25       145799.0      1
           XR         132422.0      2
X-MC       X40N       120203.0      1
X-LC       XY25       111497.0      3
M-LC       XR         107132.0      1
H-MC       XR          99440.0      1
M-LC       X5K         92620.0      1
L-SC       OX40N       89888.0      1
X-SC       X40         79049.0      1
H-LC       X200        74909.0      1
H-MC       OX40N       72655.0      1
H-SC       MH          71080.0      1
L-MC       XR          59816.0      1
L-LC       XY25        42672.0      1
M-LC       XY25        41494.0      1
           XY24        27816.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
